In [1]:
# Conexion SQL
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from datetime import datetime

In [2]:
#CONEXION A SERVIDOR Y TABLA
server = 'MSLABBD01\VTAPREPROD'
database = 'Ventas'
user_db = 'vtausr'
password_db = 'vtausr'

engine = create_engine(f'mssql+pymssql://{user_db}:{password_db}@{server}/{database}?charset=utf8')

In [3]:
#Periodo que analizamos Historial de Facturacion
Periodo = 6             ##Elegimos un periodo de 6 meses
date = datetime.now()
Fin_year_month = date.strftime('%Y%m')
Inicio_year_month= int(Fin_year_month) -  Periodo
print ("AnioMes Inicio:",Inicio_year_month)
print ("AnioMes Fin:",Fin_year_month)

AnioMes Inicio: 202105
AnioMes Fin: 202111


In [4]:
#Obtencion de Tabla DIVERSIDAD DE CLIENTES

query_Tabla_Diversidad = f'select X.ClaArticulo, COUNT(X.ClaArticulo) as Diversidad, X.ClaGiroComercial\
                             FROM (select distinct  est.ClaCliente , est.ClaArticulo, cta.ClaGiroComercial\
                             from VtaSch.VtaTraEstVenXDisVw est (nolock) inner join VtaSch.VtaCatClienteCuentaVw\
                             cta (nolock) on est.ClaCliente = cta.ClaClienteCuenta\
                             where AnioMes > {Inicio_year_month} and AnioMes <= {Fin_year_month} and ClaConsignado = 0 and TotKil > 0) X GROUP BY X.ClaArticulo, X.ClaGiroComercial'
Tabla_Diversidad = pd.read_sql(sql = query_Tabla_Diversidad, con=engine)
Tabla_Diversidad

,ClaArticulo,Diversidad,ClaGiroComercial
0,3,2,101
1,4,29,101
2,5,39,101
3,6,30,101
4,7,48,101
...,...,...,...
20131,259111,1,299
20132,560370,1,299
20133,563759,1,299
20134,563760,1,299


In [5]:
#Obtencion de Tabla RELACION (Articulo_In ->Articulo_Relacionado)

Tabla_Relacion = pd.DataFrame({"Articulo":[], "ArticuloRelacionado":[], "Prioridad":[]})

for Art, Giro in zip( Tabla_Diversidad ["ClaArticulo"], Tabla_Diversidad ["ClaGiroComercial"]):
    query_Clientes_compraron_Art = f'select distinct est.ClaCliente from VtaSch.VtaTraEstVenXDisVw est (nolock) \
                                    inner join VtaSch.VtaCatClienteCuentaVw cta (nolock) on est.ClaCliente = cta.ClaClienteCuenta\
                                    where AnioMes >= {Inicio_year_month} and AnioMes <= {Fin_year_month} and  est.ClaArticulo = {Art} and\
                                    cta.ClaGiroComercial = {Giro}\
                                    and ClaConsignado = 0 and TotKil > 0'
    
    Clientes_compraron_Art =  pd.read_sql(sql = query_Clientes_compraron_Art, con=engine)  

    lista_Clientes =  tuple(Clientes_compraron_Art["ClaCliente"])

    if len(lista_Clientes) == 1:
        lista_Clientes = f"({lista_Clientes[0]})"
    
    #print (lista_Clientes, '\n')

    query_Art_de_Cliente = f'SELECT top 10 Y.ClaArticulo, COUNT(Y.ClaArticulo) as NoVecesComprado FROM\
                                (select est.ClaArticulo , est.ClaCliente\
                                from VtaSch.VtaTraEstVenXDisVw est (nolock)\
                                inner join VtaSch.VtaCatClienteCuentaVw cta (nolock) on est.ClaCliente = cta.ClaClienteCuenta\
                                where AnioMes >= 202104  and AnioMes <= 202110 \
                                and est.ClaArticulo != {Art}\
                                and ClaConsignado = 0\
                                and TotKil > 0 \
                                and est.ClaCliente in {lista_Clientes}) Y\
                                GROUP BY Y.ClaArticulo\
                                order by NoVecesComprado desc'
    Art_de_Cliente =  pd.read_sql(sql = query_Art_de_Cliente, con=engine)
    Art_de_Cliente.rename(columns={"ClaArticulo": "ArticuloRelacionado", "NoVecesComprado":"Prioridad"}, inplace=True)   
    Art_de_Cliente["Articulo"] = Art   

    Tabla_Relacion = Tabla_Relacion.append(Art_de_Cliente, ignore_index=True)  
    #if (len (Tabla_Relacion) > 100): break   #...break para pruebas


Tabla_Relacion

##Falta intruccion de grabar a BD las Tabla_Diversidad, Tabla_Relacion


,Articulo,ArticuloRelacionado,Prioridad
0,3.0,13.0,4.0
1,3.0,5.0,4.0
2,3.0,500067.0,4.0
3,3.0,530890.0,4.0
4,3.0,15.0,3.0
...,...,...,...
105,17.0,3436.0,63.0
106,17.0,536641.0,60.0
107,17.0,3437.0,57.0
108,17.0,261577.0,57.0
